In [ ]:
my_api_key = ""

In [ ]:
pip install openai

In [ ]:
import openai
import os
from openai import OpenAI
import pandas as pd

In [ ]:
prompts = pd.read_csv("/content/drive/MyDrive/css summer 2024/stanford/Rating Tweets - Sheet1.csv")

In [ ]:
messages = prompts.iloc[:,3]
print(messages)

0      A better April Fool's prank would have been Ke...
1      Okay last i checked there was 30,000 deaths an...
2      <USER> I mean technically millions of possible...
3      @el_budget I wore a shining blue leather pants...
4                                    @BreeDaAuraGod_ 😂😂😂
                             ...                        
244    @nascarwoman68 Funds to sue them since our con...
245    @NDCZ Same here. I went through stages on what...
246    @RileyDoghouse77 @AtheistRepublic I don’t feel...
247    Why babies ain’t already 9 months old when the...
248              deff did not stock up enough on seltzer
Name: message, Length: 249, dtype: object


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.environ["OPENAI_API_KEY"] = my_api_key
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

user_content = "What is the meaning of life?"
system_content = "You are a helpful assistant."

response = api_call(client, system_content, user_content)
print(response)

The meaning of life is a deeply philosophical question that can vary greatly depending on individual beliefs and perspectives. Some people find meaning in their connections with others, their pursuit of personal growth, or their contributions to society. Others may find meaning through spiritual or religious beliefs. Ultimately, the search for meaning is a personal journey that each individual must navigate on their own.


In [ ]:
for message in messages:
  user_content = message
  system_content = "You are presented with the following social media post. On a scale from 0 to 100, please indicate how you would feel, as a social media user, after seeing the social media post, of (1) if you would feel satisfied with yourself, (2) if you have a positive attitude towards yourself, and (3) if you would feel that you have a number of good qualities. Please just output three numbers, separated by a comma. "

  print(message)
  response = api_call(client, system_content, user_content)
  print(response)
  print("--------------------------------------------------------------")

In [ ]:
def api_call(client, system_content, user_content):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]
  )
  return response.choices[0].message.content

In [ ]:
### system prompt generation (personas)
# how do you get the desired weights?
ages = ['15-19', '20-24', '25-29', '30-34'] #4
marital_statuses = ['never married', 'married', 'separated', 'widowed', 'divorced'] #5
races = ['White', 'Black or African-American', 'American Indian', 'Asian', 'Native Hawaiian', 'Hispanic or Latino', 'Other race'] #7
sexes = ['male', 'female'] #2
educations = ['a high school diploma', 'some college education but no diploma', 'a Bachelor\'s degree', 'a Master\'s degree', 'a PhD'] #5
incomes = ['$30,000', '$50,000', '$80,000', '$100,000', '$150,000'] #5
mental_healths = ['neuroticism', 'no neuroticism'] #2

iteration = 0

profiles = []

for age in ages:
  for marital_status in marital_statuses:
    for race in races:
      for sex in sexes:
        for education in educations:
          for income in incomes:
            for mental_health in mental_healths:
              system_prompt = f"You are a {age} year-old, {marital_status}, {race} {sex} with {education} making {income} per year, living in the United States. In regards to mental health you have {mental_health}."
              profiles.append(system_prompt)
              iteration = iteration + 1
              #if (iteration % 100 == 0):
                #print(iteration, system_prompt)

In [ ]:
### user_prompt for feelings and classification
user_prompt = """
Provide responses from this person’s perspective.
BEFORE: How do you currently feel about yourself regarding the following categories (answer on a scale from 1-5, 1 = bad, 5 = great): attractiveness and beauty, personal relationships, social life, career success, income and wealth, and health and fitness.
AFTER: Upon viewing the following social media post, how would you feel about yourself regarding the following categories (answer on a scale from 1-5, 1 = bad, 5 = great): attractiveness and beauty, personal relationships, social life, career success, income and wealth, and health and fitness.
CLASSIFICATION: As a researcher, which of the following categories does the post fit into: body image, achievements, relationships, travel / luxury, social justice.
"""